In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as reader
import datetime as dt
import tensorflow as tf
import math
from keras.layers import LSTM, GRU, SimpleRNN
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Train.csv')

In [ ]:
df

,Date,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price
0,2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000
1,2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169
2,2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191
3,2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956
4,2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2018-08-31,Stock 100,810.402398,953.465125,1763.867523,1014.718528,957.998121,675.953821,102.346000,21.785595,1016713,101.747952
14996,2018-09-28,Stock 100,811.620957,443.679899,1255.300856,392.059390,365.958118,815.913721,-301.200669,0.000000,1042638,102.039499
14997,2018-10-31,Stock 100,739.371050,423.172140,1162.543189,478.844995,396.603049,550.561664,227.299298,56.788344,1077825,101.126752
14998,2018-11-30,Stock 100,741.898496,286.891422,1028.789919,347.793349,358.229637,176.189978,19.405005,5.183536,1003161,102.020776


Lets set up the index to datetime

In [ ]:
df_d = df

In [ ]:
df_d['Date'] = pd.to_datetime(df_d['Date'])

In [ ]:
df_d.set_index('Date', inplace=True)

In [ ]:
df_d

,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price
Date,,,,,,,,,,,
2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000
2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169
2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191
2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956
2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907
...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,Stock 100,810.402398,953.465125,1763.867523,1014.718528,957.998121,675.953821,102.346000,21.785595,1016713,101.747952
2018-09-28,Stock 100,811.620957,443.679899,1255.300856,392.059390,365.958118,815.913721,-301.200669,0.000000,1042638,102.039499
2018-10-31,Stock 100,739.371050,423.172140,1162.543189,478.844995,396.603049,550.561664,227.299298,56.788344,1077825,101.126752


##EDA

In [ ]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_d.describe()

,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000
mean,547.519819,549.476881,1096.996699,604.578098,634.164054,552.653822,0.802641,17.410165,1.050185e+06,101.401818
std,260.201093,260.453313,366.527469,264.000561,292.592581,259.456550,175.843566,25.893009,2.907637e+04,10.298443
min,100.068495,100.199767,214.814273,76.785192,74.296869,100.124239,-493.404528,0.000000,1.000003e+06,68.509892
25%,319.351022,324.739067,832.080518,406.884837,416.449707,328.253976,-109.777787,0.000000,1.024837e+06,95.737983
50%,544.731700,544.898627,1094.930216,567.370669,589.586598,555.204989,2.531044,0.620391,1.050500e+06,100.665047
75%,771.242141,775.209146,1360.909341,773.418198,810.304473,777.533156,113.132409,27.961753,1.075450e+06,106.039544
max,999.907125,999.805167,1992.436554,1556.094765,1885.219463,999.992530,499.025639,141.982375,1.099990e+06,143.008924


In [ ]:
years = df_d.groupby("Date")["price"].mean().reset_index()
px.scatter(years,x="Date", y="price").show()

*Feature engineering*

In [ ]:
df_d['current_ratio'] = df_d['current_assets']/df_d['current_liabilities']
df_d['divident_payout'] = df_d['dividend']/df_d['net_income']
df_d['debt_equity_ratio'] = df_d['liabilities']/df_d['equity']
df_d['ROE'] = df_d['net_income']/df_d['equity']
df_d['divident_yield'] = df_d['dividend']/df_d['price']
df_d['EPS'] = df_d['net_income']/df_d['shares_outstanding']
df_d['ROA'] = df_d['net_income']/df_d['total_assets']

In [ ]:
df_d.describe()

,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price,current_ratio,divident_payout,debt_equity_ratio,ROE,divident_yield,EPS,ROA
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000
mean,547.519819,549.476881,1096.996699,604.578098,634.164054,552.653822,0.802641,17.410165,1.050185e+06,101.401818,0.971565,0.126680,1.404490,0.002094,0.173384,7.414362e-07,0.001205
std,260.201093,260.453313,366.527469,264.000561,292.592581,259.456550,175.843566,25.893009,2.907637e+04,10.298443,0.136622,0.126696,1.306985,0.560573,0.259553,1.676183e-04,0.206384
min,100.068495,100.199767,214.814273,76.785192,74.296869,100.124239,-493.404528,0.000000,1.000003e+06,68.509892,0.769250,-0.000000,0.104000,-4.067149,0.000000,-4.856426e-04,-1.725827
25%,319.351022,324.739067,832.080518,406.884837,416.449707,328.253976,-109.777787,0.000000,1.024837e+06,95.737983,0.851212,-0.000000,0.583627,-0.213984,0.000000,-1.045440e-04,-0.102822
50%,544.731700,544.898627,1094.930216,567.370669,589.586598,555.204989,2.531044,0.620391,1.050500e+06,100.665047,0.953219,0.201394,0.993961,0.004635,0.006240,2.433823e-06,0.002306
75%,771.242141,775.209146,1360.909341,773.418198,810.304473,777.533156,113.132409,27.961753,1.075450e+06,106.039544,1.081055,0.250937,1.703591,0.222180,0.276877,1.080603e-04,0.107523
max,999.907125,999.805167,1992.436554,1556.094765,1885.219463,999.992530,499.025639,141.982375,1.099990e+06,143.008924,1.249999,0.300000,9.590760,3.957818,1.776392,4.890101e-04,1.506926


In [ ]:
# column_list = df['shares_outstanding'].to_list()
# df_sample = pd.DataFrame({'shares_outstanding': column_list})

# min_value = df_sample['shares_outstanding'].min()
# max_value = df_sample['shares_outstanding'].max()

# df_d['n_shares'] = ((df_sample['shares_outstanding'] - min_value) / (max_value - min_value)) * 1000

In [ ]:
df_d

,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price,current_ratio,divident_payout,debt_equity_ratio,ROE,divident_yield,EPS,ROA
Date,,,,,,,,,,,,,,,,,,
2006-07-31,Stock 1,625.898686,925.086518,1550.985204,675.789905,555.931329,548.488451,251.282553,68.184030,1040831,100.000000,1.215600,0.271344,0.676584,0.271631,0.681840,0.000241,0.162015
2006-08-31,Stock 1,407.387467,903.249093,1310.636561,1023.110983,1295.186240,513.446432,207.576940,42.232672,1046649,101.924169,0.789933,0.203456,0.451024,0.229811,0.414354,0.000198,0.158379
2006-09-29,Stock 1,464.069891,602.668832,1066.738722,709.711673,598.719338,503.519871,133.719700,28.835722,1094158,103.166191,1.185383,0.215643,0.770025,0.221879,0.279507,0.000122,0.125354
2006-10-31,Stock 1,855.854937,391.375369,1247.230306,649.979564,733.594339,596.006397,-76.881818,0.000000,1018574,104.478956,0.886020,-0.000000,2.186788,-0.196440,0.000000,-0.000075,-0.061642
2006-11-30,Stock 1,453.954969,956.490566,1410.445534,1115.594551,1152.274711,965.939247,-309.204301,0.000000,1036293,104.160907,0.968167,-0.000000,0.474605,-0.323270,0.000000,-0.000298,-0.219225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,Stock 100,810.402398,953.465125,1763.867523,1014.718528,957.998121,675.953821,102.346000,21.785595,1016713,101.747952,1.059207,0.212862,0.849955,0.107341,0.214113,0.000101,0.058024
2018-09-28,Stock 100,811.620957,443.679899,1255.300856,392.059390,365.958118,815.913721,-301.200669,0.000000,1042638,102.039499,1.071323,-0.000000,1.829294,-0.678869,0.000000,-0.000289,-0.239943
2018-10-31,Stock 100,739.371050,423.172140,1162.543189,478.844995,396.603049,550.561664,227.299298,56.788344,1077825,101.126752,1.207366,0.249840,1.747211,0.537132,0.561556,0.000211,0.195519


*Normalization (Method - MinMax)*

In [ ]:
numeric_cols = df_d.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_d.select_dtypes('object').columns.tolist()

In [ ]:
print(numeric_cols)

['liabilities', 'equity', 'total_assets', 'current_assets', 'current_liabilities', 'total_revenue', 'net_income', 'dividend', 'shares_outstanding', 'price', 'current_ratio', 'divident_payout', 'debt_equity_ratio', 'ROE', 'divident_yield', 'EPS', 'ROA']


In [ ]:
print(categorical_cols)

['Stock']


In [ ]:
numeric_cols.remove('price')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(df_d[numeric_cols])

In [ ]:
df_d[numeric_cols] = scaler.transform(df_d[numeric_cols])

In [ ]:
df_d

,Stock,liabilities,equity,total_assets,current_assets,current_liabilities,total_revenue,net_income,dividend,shares_outstanding,price,current_ratio,divident_payout,debt_equity_ratio,ROE,divident_yield,EPS,ROA
Date,,,,,,,,,,,,,,,,,,
2006-07-31,Stock 1,0.584361,0.916943,0.751662,0.404922,0.265961,0.498255,0.750367,0.480229,0.408333,100.000000,0.928447,0.904481,0.060356,0.540660,0.383834,0.745976,0.583973
2006-08-31,Stock 1,0.341527,0.892668,0.616454,0.639708,0.674181,0.459314,0.706328,0.297450,0.466521,101.924169,0.043024,0.678186,0.036580,0.535449,0.233256,0.701755,0.582849
2006-09-29,Stock 1,0.404519,0.558544,0.479249,0.427853,0.289589,0.448283,0.631908,0.203094,0.941672,103.166191,0.865594,0.718811,0.070206,0.534461,0.157346,0.623663,0.572633
2006-10-31,Stock 1,0.839913,0.323670,0.580785,0.387474,0.364067,0.551061,0.419700,0.000000,0.185734,104.478956,0.242893,0.000000,0.219547,0.482333,0.000000,0.420830,0.514789
2006-11-30,Stock 1,0.393278,0.951852,0.672601,0.702226,0.595264,0.962157,0.185605,0.000000,0.362947,104.160907,0.413766,0.000000,0.039065,0.466529,0.000000,0.192137,0.466043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,Stock 100,0.789401,0.948488,0.871419,0.634035,0.487984,0.639904,0.600295,0.153439,0.167122,101.747952,0.603137,0.709542,0.078631,0.520188,0.120533,0.601554,0.551805
2018-09-28,Stock 100,0.790756,0.381812,0.585325,0.213123,0.161057,0.795438,0.193670,0.000000,0.426405,102.039499,0.628339,0.000000,0.181863,0.422217,0.000000,0.201876,0.459634
2018-10-31,Stock 100,0.710464,0.359016,0.533144,0.271789,0.177979,0.500559,0.726201,0.399968,0.778321,101.126752,0.911320,0.832799,0.173211,0.573745,0.316122,0.714644,0.594337


In [ ]:
pred = df.loc[:, ~df_d.columns.isin(['price', 'Stock', 'Date'])]
targ = df["price"]

In [ ]:
targ

Date
2006-07-31    100.000000
2006-08-31    101.924169
2006-09-29    103.166191
2006-10-31    104.478956
2006-11-30    104.160907
                 ...    
2018-08-31    101.747952
2018-09-28    102.039499
2018-10-31    101.126752
2018-11-30    102.020776
2018-12-31    102.594684
Name: price, Length: 15000, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( pred, targ, test_size=0.33, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
# SVM Regressor
svm_reg = SVR()
svm_reg.fit(X_train, y_train)
y_pred_svm = svm_reg.predict(X_test)
rmse_svm = np.sqrt(mean_squared_error(y_test, y_pred_svm))
print("SVM RMSE:", rmse_svm)

# KNN Regressor
knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train, y_train)
y_pred_knn = knn_reg.predict(X_test)
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
print("KNN RMSE:", rmse_knn)

# Random Forest Regressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
y_pred_rf = rf_reg.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Random Forest RMSE:", rmse_rf)


#XGBoost
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("XGB RMSE:", rmse)

SVM RMSE: 10.187619340774981
KNN RMSE: 11.163666473928254
Random Forest RMSE: 7.881654061733979
XGB RMSE: 8.26018576571818


In [ ]:
df_d.columns

Index(['Stock', 'liabilities', 'equity', 'total_assets', 'current_assets',
       'current_liabilities', 'total_revenue', 'net_income', 'dividend',
       'shares_outstanding', 'price', 'current_ratio', 'divident_payout',
       'debt_equity_ratio', 'ROE', 'divident_yield', 'EPS', 'ROA'],
      dtype='object')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.layers import Dense

data = df_d

features = [ 'liabilities', 'equity', 'total_assets', 'current_assets',
       'current_liabilities', 'total_revenue', 'net_income', 'dividend',
       'shares_outstanding', 'current_ratio', 'divident_payout',
       'debt_equity_ratio', 'ROE', 'divident_yield', 'EPS', 'ROA']
target = 'price'



X = data[features].values
y = data[target].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the MLP model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

# Evaluate the model
mse = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error:', mse)

# Make predictions
predictions = model.predict(X_test)

# Flatten the predictions
predictions = predictions.flatten()

# Calculate the minimum and maximum values of the target variable
y_min = np.min(y)
y_max = np.max(y)

# Perform inverse normalization on the predicted values
predictions = predictions * (y_max - y_min) + y_min

# Compare actual and predicted values
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(comparison)




Epoch 1/20
750/750 [==============================] - 6s 6ms/step - loss: 1062.5078
Epoch 2/20
750/750 [==============================] - 3s 4ms/step - loss: 122.0048
Epoch 3/20
750/750 [==============================] - 2s 3ms/step - loss: 110.6508
Epoch 4/20
750/750 [==============================] - 2s 2ms/step - loss: 103.5176
Epoch 5/20
750/750 [==============================] - 2s 2ms/step - loss: 99.1945
Epoch 6/20
750/750 [==============================] - 2s 3ms/step - loss: 96.3208
Epoch 7/20
750/750 [==============================] - 2s 3ms/step - loss: 93.2381
Epoch 8/20
750/750 [==============================] - 2s 3ms/step - loss: 90.0854
Epoch 9/20
750/750 [==============================] - 2s 3ms/step - loss: 86.4426
Epoch 10/20
750/750 [==============================] - 2s 2ms/step - loss: 83.8669
Epoch 11/20
750/750 [==============================] - 2s 2ms/step - loss: 81.0731
Epoch 12/20
750/750 [==============================] - 2s 3ms/step - loss: 79.6521
Epoch 13

We see that deep neural networks are not performing that well as compared to ensemble regressors